In [1]:
import torch
import os

from src.rl.evaluators.evaluator_dqn import EvaluatorDQN
from src.rl.evaluators.evaluator_c51 import EvaluatorC51
from src.rl.evaluators.evaluator_qr import EvaluatorQR
from src.rl.evaluators.evaluator_iqn import EvaluatorIQN
from src.rl.evaluators.evaluator_fqf import EvaluatorFQF
from src.rl.evaluators.evaluator_ddpg import EvaluatorDDPG
from src.rl.evaluators.evaluator_td3 import EvaluatorTD3
from src.rl.evaluators.evaluator_reinforce import EvaluatorREINFORCE

In [2]:
%load_ext autoreload
%autoreload 2

# Random Baseline

In the following cell, we create random predictions to be used as a lower bound baseline. We get random baselines for the average return on the development set, as well as predictions on the test set to be evaluated via the MIND competition.

In [ ]:
from csv import writer
from io import StringIO
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

import src.constants as constants
from src.common_utils import read_pickled_data

In [ ]:
# Change the values, if wanted
seeds = [7, 42]

In [ ]:
eval_data = read_pickled_data([
    constants.DEV_PATH,
    "preprocessed",
    "behaviors.pkl"
])
model_dir = os.path.join(
    constants.MODELS_PATH,
    "Baseline"
)
gamma = 0.9

In [ ]:
for seed in seeds:
    results = StringIO()
    csv_writer = writer(results)
    columns = ["checkpoint", "mean_return"]
    csv_writer.writerow(columns)

    pred_dir_name = "predictions" if seed is None else f"predictions_{seed}"
    pred_dir = os.path.join(
        model_dir,
        pred_dir_name
    )
    if not os.path.exists(pred_dir):
        os.makedirs(pred_dir)
    #! Set seed
    np.random.seed(seed)

    returns = []

    for row in tqdm(eval_data.itertuples(), total=len(eval_data)):
        shown_news = row.shown_news
        clicked_news = set(row.clicked_news)

        # Randomly order candidates
        np.random.shuffle(shown_news)
        
        # Compute return
        G = 0
        for t, news_id in enumerate(shown_news):
            reward = 0
            if news_id in clicked_news:
                reward = 1
            G += ((gamma**t) * reward)
        returns.append(G)

    # Compute average return
    mean_return = np.array(returns).mean()
    csv_writer.writerow(["Baseline", mean_return])

    print(f"[INFO] writing evaluation results file to {pred_dir}")
    results.seek(0)
    data_eval_results = pd.read_csv(results)
    data_eval_results.to_csv(
        os.path.join(pred_dir, "eval_results.txt"),
        sep='\t',
        index=False,
        header=True
    )

### Development Set

We compute the average return for multiple random baselines, as well as the average over all baselines.

# Evaluation

In [3]:
development = True
model_name = "DQN-n-m-w99"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

evaluator = EvaluatorDQN(development, model_name, device, seed=42)
evaluator.set_evaluatee()
evaluator.evaluate()

[INFO] device: cuda
[INFO] preparing predictions directory
[INFO] reading config files
[INFO] model checkpoints: ['dqn_1.pth', 'dqn_2.pth', 'dqn_3.pth', 'dqn_4.pth', 'dqn_5.pth', 'dqn_6.pth', 'dqn_7.pth', 'dqn_8.pth', 'dqn_final.pth']
[INFO] preparing data and sampler
[DONE] evaluator initialized
[INFO] evaluating 'DQN-n-m-w99', checkpoint 'dqn_1.pth'


  0%|          | 0/376471 [00:00<?, ?it/s]

[RESULT] Return: 0.6574
[INFO] evaluating 'DQN-n-m-w99', checkpoint 'dqn_2.pth'


  0%|          | 0/376471 [00:00<?, ?it/s]

[RESULT] Return: 0.7120
[INFO] evaluating 'DQN-n-m-w99', checkpoint 'dqn_3.pth'


...

[RESULT] Return: 0.7780
[INFO] evaluating 'DQN-n-m-w99', checkpoint 'dqn_8.pth'


  0%|          | 0/376471 [00:00<?, ?it/s]

[RESULT] Return: 0.7787
[INFO] evaluating 'DQN-n-m-w99', checkpoint 'dqn_final.pth'


  0%|          | 0/376471 [00:00<?, ?it/s]

[RESULT] Return: 0.7799
[INFO] writing evaluation results file to c:\workbench\developer\drlnrs\models\DQN-n-m-w99\predictions_42
[DONE] evaluation completed


In [4]:
development = True
model_name = "DDPG"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

evaluator = EvaluatorDDPG(development, model_name, device, seed=7)
evaluator.set_evaluatee(involved="a")
evaluator.evaluate()

[INFO] device: cuda
[INFO] preparing predictions directory
[INFO] reading config files
[INFO] model checkpoints: ['actor_1.pth', 'actor_2.pth', 'actor_3.pth', 'actor_4.pth', 'actor_5.pth', 'actor_final.pth', 'critic_1.pth', 'critic_2.pth', 'critic_3.pth', 'critic_4.pth', 'critic_5.pth', 'critic_final.pth']
[INFO] preparing data and sampler
[DONE] evaluator initialized
[INFO] evaluating 'DDPG', checkpoint '('c:\\workbench\\developer\\drlnrs\\models\\DDPG\\checkpoints_7\\actor_1.pth', 'c:\\workbench\\developer\\drlnrs\\models\\DDPG\\checkpoints_7\\critic_1.pth')'


  0%|          | 0/376471 [00:00<?, ?it/s]

[RESULT] Return: 0.5792
[INFO] evaluating 'DDPG', checkpoint '('c:\\workbench\\developer\\drlnrs\\models\\DDPG\\checkpoints_7\\actor_2.pth', 'c:\\workbench\\developer\\drlnrs\\models\\DDPG\\checkpoints_7\\critic_2.pth')'


  0%|          | 0/376471 [00:00<?, ?it/s]

[RESULT] Return: 0.5996
[INFO] evaluating 'DDPG', checkpoint '('c:\\workbench\\developer\\drlnrs\\models\\DDPG\\checkpoints_7\\actor_3.pth', 'c:\\workbench\\developer\\drlnrs\\models\\DDPG\\checkpoints_7\\critic_3.pth')'


...

[RESULT] Return: 0.5971
[INFO] evaluating 'DDPG', checkpoint '('c:\\workbench\\developer\\drlnrs\\models\\DDPG\\checkpoints_7\\actor_5.pth', 'c:\\workbench\\developer\\drlnrs\\models\\DDPG\\checkpoints_7\\critic_5.pth')'


  0%|          | 0/376471 [00:00<?, ?it/s]

[RESULT] Return: 0.6270
[INFO] evaluating 'DDPG', checkpoint '('c:\\workbench\\developer\\drlnrs\\models\\DDPG\\checkpoints_7\\actor_final.pth', 'c:\\workbench\\developer\\drlnrs\\models\\DDPG\\checkpoints_7\\critic_final.pth')'


  0%|          | 0/376471 [00:00<?, ?it/s]

[RESULT] Return: 0.6306
[INFO] writing evaluation results file to c:\workbench\developer\drlnrs\models\DDPG\predictions_7
[DONE] evaluation completed


In [6]:
development = True
model_name = "DDPG"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

evaluator = EvaluatorDDPG(development, model_name, device, seed=42)
evaluator.set_evaluatee(involved="a")
evaluator.evaluate()

[INFO] device: cuda
[INFO] preparing predictions directory
[INFO] reading config files
[INFO] model checkpoints: ['actor_1.pth', 'actor_2.pth', 'actor_3.pth', 'actor_4.pth', 'actor_5.pth', 'actor_final.pth', 'critic_1.pth', 'critic_2.pth', 'critic_3.pth', 'critic_4.pth', 'critic_5.pth', 'critic_final.pth']
[INFO] preparing data and sampler
[DONE] evaluator initialized
[INFO] evaluating 'DDPG', checkpoint '('c:\\workbench\\developer\\drlnrs\\models\\DDPG\\checkpoints_42\\actor_1.pth', 'c:\\workbench\\developer\\drlnrs\\models\\DDPG\\checkpoints_42\\critic_1.pth')'


  0%|          | 0/376471 [00:00<?, ?it/s]

[RESULT] Return: 0.6304
[INFO] evaluating 'DDPG', checkpoint '('c:\\workbench\\developer\\drlnrs\\models\\DDPG\\checkpoints_42\\actor_2.pth', 'c:\\workbench\\developer\\drlnrs\\models\\DDPG\\checkpoints_42\\critic_2.pth')'


  0%|          | 0/376471 [00:00<?, ?it/s]

[RESULT] Return: 0.6180
[INFO] evaluating 'DDPG', checkpoint '('c:\\workbench\\developer\\drlnrs\\models\\DDPG\\checkpoints_42\\actor_3.pth', 'c:\\workbench\\developer\\drlnrs\\models\\DDPG\\checkpoints_42\\critic_3.pth')'


...

[RESULT] Return: 0.6258
[INFO] evaluating 'DDPG', checkpoint '('c:\\workbench\\developer\\drlnrs\\models\\DDPG\\checkpoints_42\\actor_5.pth', 'c:\\workbench\\developer\\drlnrs\\models\\DDPG\\checkpoints_42\\critic_5.pth')'


  0%|          | 0/376471 [00:00<?, ?it/s]

[RESULT] Return: 0.6298
[INFO] evaluating 'DDPG', checkpoint '('c:\\workbench\\developer\\drlnrs\\models\\DDPG\\checkpoints_42\\actor_final.pth', 'c:\\workbench\\developer\\drlnrs\\models\\DDPG\\checkpoints_42\\critic_final.pth')'


  0%|          | 0/376471 [00:00<?, ?it/s]

[RESULT] Return: 0.6299
[INFO] writing evaluation results file to c:\workbench\developer\drlnrs\models\DDPG\predictions_42
[DONE] evaluation completed


In [7]:
development = True
model_name = "DDPG"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

evaluator = EvaluatorDDPG(development, model_name, device, seed=7)
evaluator.set_evaluatee(involved="ac")
evaluator.evaluate()

[INFO] device: cuda
[INFO] preparing predictions directory
[INFO] reading config files
[INFO] model checkpoints: ['actor_1.pth', 'actor_2.pth', 'actor_3.pth', 'actor_4.pth', 'actor_5.pth', 'actor_final.pth', 'critic_1.pth', 'critic_2.pth', 'critic_3.pth', 'critic_4.pth', 'critic_5.pth', 'critic_final.pth']
[INFO] preparing data and sampler
[DONE] evaluator initialized
[INFO] evaluating 'DDPG', checkpoint '('c:\\workbench\\developer\\drlnrs\\models\\DDPG\\checkpoints_7\\actor_1.pth', 'c:\\workbench\\developer\\drlnrs\\models\\DDPG\\checkpoints_7\\critic_1.pth')'


  0%|          | 0/376471 [00:00<?, ?it/s]

[RESULT] Return: 0.6155
[INFO] evaluating 'DDPG', checkpoint '('c:\\workbench\\developer\\drlnrs\\models\\DDPG\\checkpoints_7\\actor_2.pth', 'c:\\workbench\\developer\\drlnrs\\models\\DDPG\\checkpoints_7\\critic_2.pth')'


  0%|          | 0/376471 [00:00<?, ?it/s]

[RESULT] Return: 0.6703
[INFO] evaluating 'DDPG', checkpoint '('c:\\workbench\\developer\\drlnrs\\models\\DDPG\\checkpoints_7\\actor_3.pth', 'c:\\workbench\\developer\\drlnrs\\models\\DDPG\\checkpoints_7\\critic_3.pth')'


...

[RESULT] Return: 0.6830
[INFO] evaluating 'DDPG', checkpoint '('c:\\workbench\\developer\\drlnrs\\models\\DDPG\\checkpoints_7\\actor_5.pth', 'c:\\workbench\\developer\\drlnrs\\models\\DDPG\\checkpoints_7\\critic_5.pth')'


  0%|          | 0/376471 [00:00<?, ?it/s]

[RESULT] Return: 0.7159
[INFO] evaluating 'DDPG', checkpoint '('c:\\workbench\\developer\\drlnrs\\models\\DDPG\\checkpoints_7\\actor_final.pth', 'c:\\workbench\\developer\\drlnrs\\models\\DDPG\\checkpoints_7\\critic_final.pth')'


  0%|          | 0/376471 [00:00<?, ?it/s]

[RESULT] Return: 0.7153
[INFO] writing evaluation results file to c:\workbench\developer\drlnrs\models\DDPG\predictions_7
[DONE] evaluation completed


In [8]:
development = True
model_name = "DDPG"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

evaluator = EvaluatorDDPG(development, model_name, device, seed=42)
evaluator.set_evaluatee(involved="ac")
evaluator.evaluate()

[INFO] device: cuda
[INFO] preparing predictions directory
[INFO] reading config files
[INFO] model checkpoints: ['actor_1.pth', 'actor_2.pth', 'actor_3.pth', 'actor_4.pth', 'actor_5.pth', 'actor_final.pth', 'critic_1.pth', 'critic_2.pth', 'critic_3.pth', 'critic_4.pth', 'critic_5.pth', 'critic_final.pth']
[INFO] preparing data and sampler
[DONE] evaluator initialized
[INFO] evaluating 'DDPG', checkpoint '('c:\\workbench\\developer\\drlnrs\\models\\DDPG\\checkpoints_42\\actor_1.pth', 'c:\\workbench\\developer\\drlnrs\\models\\DDPG\\checkpoints_42\\critic_1.pth')'


  0%|          | 0/376471 [00:00<?, ?it/s]

[RESULT] Return: 0.6602
[INFO] evaluating 'DDPG', checkpoint '('c:\\workbench\\developer\\drlnrs\\models\\DDPG\\checkpoints_42\\actor_2.pth', 'c:\\workbench\\developer\\drlnrs\\models\\DDPG\\checkpoints_42\\critic_2.pth')'


  0%|          | 0/376471 [00:00<?, ?it/s]

[RESULT] Return: 0.6801
[INFO] evaluating 'DDPG', checkpoint '('c:\\workbench\\developer\\drlnrs\\models\\DDPG\\checkpoints_42\\actor_3.pth', 'c:\\workbench\\developer\\drlnrs\\models\\DDPG\\checkpoints_42\\critic_3.pth')'


...

[RESULT] Return: 0.7052
[INFO] evaluating 'DDPG', checkpoint '('c:\\workbench\\developer\\drlnrs\\models\\DDPG\\checkpoints_42\\actor_5.pth', 'c:\\workbench\\developer\\drlnrs\\models\\DDPG\\checkpoints_42\\critic_5.pth')'


  0%|          | 0/376471 [00:00<?, ?it/s]

[RESULT] Return: 0.7208
[INFO] evaluating 'DDPG', checkpoint '('c:\\workbench\\developer\\drlnrs\\models\\DDPG\\checkpoints_42\\actor_final.pth', 'c:\\workbench\\developer\\drlnrs\\models\\DDPG\\checkpoints_42\\critic_final.pth')'


  0%|          | 0/376471 [00:00<?, ?it/s]

[RESULT] Return: 0.7182
[INFO] writing evaluation results file to c:\workbench\developer\drlnrs\models\DDPG\predictions_42
[DONE] evaluation completed
